In [1]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset
import transformers
import accelerate
import torch



In [2]:
# Load pre-trained model and tokenizer
model_name = "gpt2"  # You can also use "gpt2-medium", "gpt2-large", etc.
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

In [3]:
from datasets import load_dataset

# Load your dataset (make sure the file exists in this path)
dataset = load_dataset('text', data_files=r'C:\Users\T H I N K P A D\desktop\GPT_2\my_dataset.txt')
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 11
    })
})


In [4]:
from datasets import load_dataset
from transformers import GPT2Tokenizer

# Load your dataset
dataset = load_dataset('text', data_files=r'C:\Users\T H I N K P A D\desktop\GPT_2\my_dataset.txt')

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Set the padding token
tokenizer.pad_token = tokenizer.eos_token

# Define the tokenization function
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

# Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Print the tokenized dataset
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 11
    })
})


In [12]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset

# Load your dataset
dataset = load_dataset('text', data_files=r'C:\Users\T H I N K P A D\desktop\GPT_2\my_dataset.txt')

# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token  # Set padding token
tokenizer.pad_token_id = tokenizer.eos_token_id  # Set pad token id
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Tokenization function
def tokenize_function(examples):
    tokenized = tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=512,
        return_tensors='pt'  # Return PyTorch tensors
    )
    tokenized['labels'] = tokenized['input_ids']  # Set labels to input_ids
    return tokenized

# Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=10,       # Log every 10 steps
)

# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'] if 'test' in tokenized_datasets else None,
)

# Start training
trainer.train()

Map:   0%|          | 0/11 [00:00<?, ? examples/s]

Step,Training Loss
10,4.198600


TrainOutput(global_step=18, training_loss=2.6748187277052136, metrics={'train_runtime': 309.547, 'train_samples_per_second': 0.107, 'train_steps_per_second': 0.058, 'total_flos': 8622637056000.0, 'train_loss': 2.6748187277052136, 'epoch': 3.0})

In [13]:
model.save_pretrained("./gpt2-finetuned")
tokenizer.save_pretrained("./gpt2-finetuned")

('./gpt2-finetuned\\tokenizer_config.json',
 './gpt2-finetuned\\special_tokens_map.json',
 './gpt2-finetuned\\vocab.json',
 './gpt2-finetuned\\merges.txt',
 './gpt2-finetuned\\added_tokens.json')

In [23]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the fine-tuned model
model = GPT2LMHeadModel.from_pretrained("./gpt2-finetuned")
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2-finetuned")

# Set the pad token id
tokenizer.pad_token = tokenizer.eos_token  # Set padding token
tokenizer.pad_token_id = tokenizer.eos_token_id  # Set pad token id

# Generate text
prompt = "As the sun set over the horizon, "  # A more specific prompt
inputs = tokenizer.encode(prompt, return_tensors="pt")

# Create attention mask
attention_mask = (inputs != tokenizer.pad_token_id).long()  # Create attention mask

# Generate output
outputs = model.generate(inputs, attention_mask=attention_mask, max_length=100, num_return_sequences=1)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


As the sun set over the horizon,  the sun was still shining.  The sun was still shining.  The sun was still shining.  The sun was still shining.  The sun was still shining.  The sun was still shining.  The sun was still shining.  The sun was still shining.  The sun was still shining.  The sun was still shining.  The sun was still shining.  The sun
